### 문제 
- 사각형으로 구성된 집에 쥐와 구멍의 좌표가 주워진다.
  - 집의 입력은 1000개 이하의 코너 포인트로 주어진다.
- 구멍엔 최대 `k`마리의 쥐가 들어갈 수 있으며, 쥐는 자신이 있는 위치에서 구명이 보여야만 들어갈 수 있다.

### 접근
- 집의 경계를 선분으로 만들고 $M$ 마리의 쥐의 좌표에서 각각 $H$ 개의 구멍으로 선분 $\overline{m_ih_j}$ 을 그었을 때,\
이 선분이 집안의 어떤 벽과도 교차하지 않으면 $m_i$와 $h_j$의 용량이 k인 간선을 연결하는 것으로 유량 그래프를 구성한다.
- 집의 코너가 반시계 방향으로 주워지므로 볼록껍질을 통해 단순다각형을 만든것이 집의 모양이 될 것이다.
  - n이 1이상이라는 조건이 조금 걸린다. 문제에서 모든 쥐들은 엄격하게 집 내부에 있다고 하는데 그러려면 n이 3이상일것이 보장되어야 하지 않나?
  - ㄴ 볼록껍질을 할 필요가 없을수도 있다. 집의 모양은 xy축에 평행한 직교 다각형의 형태라고 했기 떄문.

In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

import collections
def edmonds(C, src, snk) :
  V = len(C)
  F = [[0] * V for _ in range(V)]
  ret = 0

  while True :
    tmp = 0
    Q = collections.deque()
    vis = [False] * V
    P = [-1] * V
    vis[src] = True
    Q.append((src, 1 << 63))

    while Q :
      front = Q.popleft()
      u, cur = front
      if u == snk :
        tmp = cur
        break
      for v in range(V) :
        if vis[v] or F[u][v] >= C[u][v] : continue
        vis[v] = True
        P[v] = u
        Q.append((v, min(cur, C[u][v] - F[u][v])))

    if P[snk] == -1: break
    ret += tmp
    p = P[snk]
    u = snk
    while p != -1 :
      F[p][u] += tmp
      F[u][p] -= tmp
      u = p
      p = P[u]
  return ret

ccw = lambda o, a, b: (a[0] - o[0]) * (b[1] - o[1]) - (a[1] - o[1]) * (b[0] - o[0])
def cross(a, b) :
  t1 = ccw(a[0], a[1], b[0]) * ccw(a[0], a[1], b[1])
  t2 = ccw(b[0], b[1], a[0]) * ccw(b[0], b[1], a[1])
  if t1 < 0 and t2 < 0 : return True
  if t1 == 0 and t2 == 0: return b[0] <= a[1] and a[0] <= b[1]
  return t1 <= 0 and t2 <= 0

def lies(a, b, p) :
  if not min(a[0], b[0]) <= p[0] <= max(a[0], b[0]) and min(a[1], b[1]) <= p[1] <= max(a[1], b[1]): return False
  return ccw(a, b, p) == 0

def sol() :
  N, K, H, M = map(int, input().split())
  V = H + M + 2
  C = [[0] * V for _ in range(V)]
  assert(N >= 4)

  house = []
  lx, ly = map(int, input().split())
  for _ in range(N-1) :
    x, y = map(int, input().split())
    house.append(((lx, ly), (x, y)))
    lx, ly = x, y
  house.append(((lx, ly), (house[0][0])))

  hole = []
  for _ in range(H) : #hole
    x, y = map(int, input().split())
    hole.append((x, y))
  
  # 구멍을 포함하는 벽을 제외한 모든 벽에 대해 교차점이 있으면 안됨
  for i in range(M) : #mice
    x, y = map(int, input().split())
    for j, h in enumerate(hole) :
      for wall in house:
        if lies(wall[0], wall[1], h) : continue # 구멍을 포함하는 벽은 예외
        line = (h, (x, y)) # 구멍과 쥐를 잇는 선분
        if cross(wall, line) : break # 교차점이 있으면 안됨
      else : 
        C[i+1][M+j+1] = 1

  # 이분 그래프 완성
  for i in range(M) :
    C[0][i+1] = 1
  
  for i in range(H) :
    C[M+i+1][V-1] = K
  
  ans = edmonds(C, 0, V-1) >= M
  print("Possible" if ans else "Impossible")

sol()

### 자력솔
- 기본적인 접근은 정확했고, 쥐와 구멍 사이의 교차점을 찾을 때, 구멍을 포함한 벽은 고려사항에서 제외시킨다는 아이디어가 하나 추가됐다.
- 이분 그래프 완성할 때 사소한 실수가 조금 있었다.